# Build a Regression Model in Keras - Final Project C

# Libraries

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import keras

Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers import Dense

In [4]:
from sklearn.model_selection import train_test_split

# Getting the data

In [5]:
!wget "https://cocl.us/concrete_data"

--2020-01-18 19:30:15--  https://cocl.us/concrete_data
Resolving cocl.us (cocl.us)... 169.48.113.194, 158.85.108.83, 158.85.108.86
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv [following]
--2020-01-18 19:30:18--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58988 (58K) [text/csv]
Saving to: ‘concrete_data.5’

concrete_data.5     100%[===================>]  57.61K  --.-KB/s    in 0.006s  

2020-01-18 19:30:18 (

In [6]:
os.getcwd()

'/resources/labs/DL0101EN'

In [7]:
os.listdir(os.getcwd())

['DL0101EN-3-2-Classification-with-Keras-py-v1.0.ipynb',
 'concrete_data.1',
 'concrete_data',
 'concrete_data.5',
 '.ipynb_checkpoints',
 'Build a Regression Model in Keras - Final Project C.ipynb',
 'DL0101EN-1-1-Forward-Propgation-py-v1.0.ipynb',
 'concrete_data.3',
 'Build a Regression Model in Keras - Final Project B.ipynb',
 'DL0101EN-3-1-Regression-with-Keras-py-v1.0.ipynb',
 'Build a Regression Model in Keras - Final Project A.ipynb',
 'DL0101EN-4-1-Convolutional-Neural-Networks-with-Keras-py-v1.0.ipynb',
 'concrete_data.2',
 'concrete_data.4',
 'classification_model.h5']

In [8]:
df = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
Cement                1030 non-null float64
Blast Furnace Slag    1030 non-null float64
Fly Ash               1030 non-null float64
Water                 1030 non-null float64
Superplasticizer      1030 non-null float64
Coarse Aggregate      1030 non-null float64
Fine Aggregate        1030 non-null float64
Age                   1030 non-null int64
Strength              1030 non-null float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


# Exploring the data

In [9]:
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [10]:
df.tail()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77
1029,260.9,100.5,78.3,200.6,8.6,864.5,761.5,28,32.40


In [11]:
df.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [12]:
df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


### Splitting target from predictors

In [13]:
df_columns = df.columns
df_predictors = df[df_columns[df_columns != 'Strength']]
df_predictors

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360
...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28


In [14]:
df_target = df['Strength']
df_target

0       79.99
1       61.89
2       40.27
3       41.05
4       44.30
        ...  
1025    44.28
1026    31.18
1027    23.70
1028    32.77
1029    32.40
Name: Strength, Length: 1030, dtype: float64

In [ ]:
n_cols = df_predictors.shape[1]
n_cols

In [26]:
### Normalizing predictors

In [16]:
df_predictors_normalized = (df_predictors - df_predictors.mean()) / df_predictors.std()
df_predictors_normalized

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069
...,...,...,...,...,...,...,...,...
1025,-0.045623,0.487998,0.564271,-0.092126,0.451190,-1.322363,-0.065861,-0.279597
1026,0.392628,-0.856472,0.959602,0.675872,0.702285,-1.993711,0.496651,-0.279597
1027,-1.269472,0.759210,0.850222,0.521336,-0.017520,-1.035561,0.080068,-0.279597
1028,-1.168042,1.307430,-0.846733,-0.279443,0.852942,0.214537,0.191074,-0.279597


# B. Normalize the data (5 marks)

- Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df_predictors_normalized, df_target, test_size=0.3)

### Defining the model

In [18]:
def regression_model():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [19]:
model = regression_model()

### Fitting the model 50 time

In [20]:
mean_squared_error = []

for i in range(0, 50):
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100)
    scores = model.evaluate(X_test, y_test)
    print("\nMean Squared Error " + str(i) + ": {}".format(scores))
    mean_squared_error.append(scores)

Train on 721 samples, validate on 309 samples
Epoch 1/100
721/721 [==============================] - 0s 405us/step - loss: 1536.2215 - val_loss: 1619.3918
Epoch 2/100
721/721 [==============================] - 0s 69us/step - loss: 1518.4538 - val_loss: 1601.6050
Epoch 3/100
721/721 [==============================] - 0s 81us/step - loss: 1500.5638 - val_loss: 1583.7171
Epoch 4/100
721/721 [==============================] - 0s 142us/step - loss: 1482.0662 - val_loss: 1565.1870
Epoch 5/100
721/721 [==============================] - 0s 93us/step - loss: 1463.3315 - val_loss: 1545.4663
Epoch 6/100
721/721 [==============================] - 0s 77us/step - loss: 1443.6618 - val_loss: 1525.5762
Epoch 7/100
721/721 [==============================] - 0s 97us/step - loss: 1423.4624 - val_loss: 1504.9188
Epoch 8/100
721/721 [==============================] - 0s 92us/step - loss: 1402.9176 - val_loss: 1482.7287
Epoch 9/100
721/721 [==============================] - 0s 89us/step - loss: 1381.1063 - 

### Checking the list of Mean Squared Errors derived from the loop

In [21]:
len(mean_squared_error)

50

In [22]:
mse = pd.DataFrame(mean_squared_error, columns=["MSE"])
mse.head()

,MSE
0,206.658663
1,114.405340
2,70.690405
3,55.508869
4,49.816567


In [23]:
mse.tail()

,MSE
45,39.450306
46,39.462192
47,39.499640
48,39.299357
49,39.305917


### Mean of the Mean Squared Errors

In [24]:
np.mean(mse)

MSE    46.400365
dtype: float64

### Standard deviation of the Mean Squared Errors

In [25]:
np.std(mse)

MSE    25.57675
dtype: float64

# Q: How does the mean of the mean squared errors compare to that from Step B?

# A: Seems like the MSE related stats are looking worse after changing from 50 to 100 Epochs - a drawback 

### Thank you kindly for your time spend on reviewing my notebook

### Cheers, Konrad